In [1]:
import autograd.numpy as np
import autograd.scipy as sp
from autograd.scipy import special
from autograd import grad, hessian, hessian_vector_product
import matplotlib.pyplot as plt
from copy import deepcopy

from valez_finite_VI_lib import *
from generic_optimization_lib import *

In [2]:
def check_approx_eq(x, y, tol=1e-12):
    return np.max(np.abs(x - y)) < tol

In [7]:
np.random.seed(12321) # this is a seed where VI works well

alpha = 10 # IBP parameter

Num_samples = 5000 # sample size
D = 2 # dimension
# so X will be a N\times D matrix

sigma_A = 100

sigma_eps = .1 # variance of noise

K_inf = 3 # take to be large for a good approximation to the IBP

Pi, Z, mu, A, X = generate_data(Num_samples, D, K_inf, sigma_A, sigma_eps)

K_approx = deepcopy(K_inf) # variational truncation

tau, nu, phi_mu, phi_var = initialize_parameters(Num_samples, D, K_approx)
nu_init = np.round(nu * (nu >= 0.9) + nu * (nu <= 0.1)) + nu * (nu >= 0.1) * (nu <= 0.9)


In [8]:
class DataSet(object):
    def __init__(self, X, K_approx, alpha, sigma_eps, sigma_A):
        self.X = X
        self.K_approx = K_approx
        self.alpha = alpha
        self.data_shape = {'D': X.shape[1], 'N': X.shape[0] , 'K':K_approx}
        self.sigmas = {'eps': sigma_eps, 'A': sigma_A}
        #self.nu = np.empty((X.shape[0], K_approx))
        
    def wrapped_elbo(self, params):
        tau, phi_mu, phi_var, nu = \
            unpack_params(params, self.data_shape['K'], self.data_shape['D'], self.data_shape['N'])
        return compute_elbo(tau, nu, phi_mu, phi_var, self.X, self.sigmas, self.alpha)[0]
        


In [9]:
params = pack_params(deepcopy(tau), deepcopy(phi_mu), deepcopy(phi_var), deepcopy(nu))
data_set = DataSet(X, K_approx, alpha, sigma_eps, sigma_A)
data_set.wrapped_elbo(params)

-17133393.433775857

In [10]:
elbo_grad = grad(data_set.wrapped_elbo)
elbo_grad = elbo_grad(params)
print elbo_grad

[  2.33484173e+03   1.10801349e+04   9.87794660e+02 ...,  -2.07693646e+01
  -2.61558804e+00   6.02003778e+01]
